%%bash
java -Xbootclasspath/p:alpn-boot-7.1.3.v20150130.jarr -jar client_deploy.jar --service_root_url https://test-dataproc.sandbox.googleapis.com --project ${PROJECT} rejoin spark-shell-cba39d4a-72c0-4511-a110-e73ced4c9641
 


In [3]:
%%bash
gcloud beta dataproc jobs list
#gcloud beta dataproc jobs kill spark-shell-45b5a8aa-7255-40ff-af0b-0a6c99b9d91b

ls

%%bash java -Xbootclasspath/p:alpn-boot-7.1.3.v20150130.jarr -jar client_deploy.jar --service_root_url https://test-dataproc.sandbox.googleapis.com --project ${PROJECT} submit --cluster alekse

ID                                    TYPE     STATUS
b1f0efa9-dc15-4487-bc45-e9912e580214  spark    DONE
5b0d9360-a069-4044-a1fc-7601111d4f00  pyspark  DONE
Hello World.ipynb
Spark Sql Prototype.ipynb
alpn-boot-7.1.3.v20150130.jar
alpn-boot-8.1.3.v20150130.jar
client_deploy.jar
client_deploy.jar.jre1_8
datalab
publish.sh
tmp


In [2]:
import subprocess
import os

def rejoinInteractive(jobname):
    p = subprocess.Popen(['java',
                      '-Xbootclasspath/p:alpn-boot-7.1.3.v20150130.jar',
                      '-jar',
                      'client_deploy.jar',
                      '--service_root_url',
                      'https://test-dataproc.sandbox.googleapis.com',
                      '--project', 'datalab-spark',
                      'rejoin', 
                      jobname,
                      'spark-shell'], shell = False, 
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                     stdin=subprocess.PIPE, env = os.environ)
    print p.out
    print p.err

def startInteractive():
    p = subprocess.Popen(['java',
                          '-Xbootclasspath/p:alpn-boot-7.1.3.v20150130.jar',
                          '-jar',
                          'client_deploy.jar',
                          '--service_root_url',
                          'https://test-dataproc.sandbox.googleapis.com',
                          '--project', 'datalab-spark',
                          'submit', 
                          '--cluster', 
                          'alekseyv-interact', 
                          'spark-shell'], shell = False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, env = os.environ)
    print p.out
    print p.err

def executeSparkCommand(command):
    p = subprocess.Popen(['java',
                        '-Xbootclasspath/p:alpn-boot-7.1.3.v20150130.jar',
                        '-jar',
                        'client_deploy.jar',
                        '--service_root_url',
                        'https://test-dataproc.sandbox.googleapis.com',
                        '--project', 'datalab-spark',
                        'rejoin', 
                        'spark-shell-79747481-f65a-4d99-bbdf-b1067dcc5ae1', 
                        'spark-shell'], shell = False, 
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                       stdin=subprocess.PIPE, env = os.environ)
    out, err = p.communicate(command + ' \n')
    result = out + err
    return result[result.rfind('scala>', 0, result.rfind('scala>')):result.rfind('scala>')]

def executeSparkSql(sqlCommand):
      return executeSparkCommand('sqlContext.sql("'+ sqlCommand.replace('\n', '') +'").show()')

startInteractive()

OSError: [Errno 2] No such file or directory

In [1]:
import gcp
import gcp._context
import gcp._util

def jobs_list(context):
    url = 'https://dataproc.googleapis.com/v1beta1/projects/' + context.project_id + '/jobs'
    
    return gcp._util.Http.request(url, '', credentials=context.credentials)

jobs_list(gcp.Context.default())

{u'jobs': [{u'driverControlFilesUri': u'gs://dataproc-38815733-337d-4be6-bd75-9fc858a3bf57-us/google-cloud-dataproc-metainfo/9fd31970-869c-49c2-810d-28046920f1d8/jobs/b1f0efa9-dc15-4487-bc45-e9912e580214/',
   u'driverOutputResourceUri': u'gs://dataproc-38815733-337d-4be6-bd75-9fc858a3bf57-us/google-cloud-dataproc-metainfo/9fd31970-869c-49c2-810d-28046920f1d8/jobs/b1f0efa9-dc15-4487-bc45-e9912e580214/driveroutput',
   u'placement': {u'clusterName': u'cluster-1',
    u'clusterUuid': u'9fd31970-869c-49c2-810d-28046920f1d8'},
   u'reference': {u'jobId': u'b1f0efa9-dc15-4487-bc45-e9912e580214',
    u'projectId': u'datalab-spark'},
   u'sparkJob': {u'args': [u'1000'],
    u'jarFileUris': [u'file:///usr/lib/spark/lib/spark-examples.jar'],
    u'loggingConfiguration': {},
    u'mainClass': u'org.apache.spark.examples.SparkPi'},
   u'status': {u'state': u'DONE',
    u'stateStartTime': u'2015-10-02T00:17:02.832Z'},
   u'statusHistory': [{u'state': u'PENDING',
     u'stateStartTime': u'2015-10-0

In [159]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_cell_magic
def spark_sql(line, cell):
    print executeSparkSql(cell)
    
@register_cell_magic
def spark_scala(line, cell):
    print executeSparkCommand(cell)

del spark_sql,spark_scala

In [160]:
%%spark_scala
val df = sqlContext.read.json("gs://alekseyv-test/people.json")
#create temporary table people_json using org.apache.spark.sql.json options (path "gs://alekseyv-test/people.json");

scala> val df = sqlContext.read.json("gs://alekseyv-test/people.json") 
15/10/09 01:09:05 INFO org.apache.hadoop.mapred.FileInputFormat: Total input paths to process : 1
df: org.apache.spark.sql.DataFrame = [age: bigint, name: string]




In [161]:
%%spark_scala
df.registerTempTable("people")

scala> df.registerTempTable("people") 




In [163]:
%%spark_sql

select * from people

scala> sqlContext.sql("select * from people").show() 
15/10/09 01:10:53 INFO hive.ql.parse.ParseDriver: Parsing command: select * from people
15/10/09 01:10:53 INFO hive.ql.parse.ParseDriver: Parse Completed
15/10/09 01:10:53 INFO org.apache.hadoop.mapred.FileInputFormat: Total input paths to process : 1
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+





In [169]:
%%spark_sql

select sum(age) as total_age from people

scala> sqlContext.sql("select sum(age) as total_age from people").show() 
15/10/09 01:12:34 INFO hive.ql.parse.ParseDriver: Parsing command: select sum(age) as total_age from people
15/10/09 01:12:34 INFO hive.ql.parse.ParseDriver: Parse Completed
15/10/09 01:12:34 INFO org.apache.hadoop.mapred.FileInputFormat: Total input paths to process : 1
+---------+
|total_age|
+---------+
|       49|
+---------+



